In [1]:
from sklearn import svm,datasets
iris = datasets.load_iris()

In [2]:
import pandas as pd
df = pd.DataFrame(iris.data,columns=iris.feature_names)
df['flower'] = iris.target
df['flower'] = df['flower'].apply(lambda x: iris.target_names[x])
df[47:52]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
47,4.6,3.2,1.4,0.2,setosa
48,5.3,3.7,1.5,0.2,setosa
49,5.0,3.3,1.4,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor


In [3]:
#traditional method for split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(iris.data,iris.target,test_size=0.3)

In [4]:
#randomly initialize the parameter
model = svm.SVC(kernel='rbf',C=30,gamma='auto')
model.fit(X_train,y_train)
model.score(X_test, y_test)

0.9555555555555556

#### As here the value change as we refresh the train test split so we use K-fold Cross validation

# Approach 2: Use K Fold Cross validation

#### Manually try suppling models with different parameters to cross_val_score function with 5 fold cross validation

In [6]:
from sklearn.model_selection import cross_val_score

In [7]:
cross_val_score(svm.SVC(kernel='linear',C=10,gamma='auto'),iris.data, iris.target, cv=5)

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

In [8]:
cross_val_score(svm.SVC(kernel='rbf',C=10,gamma='auto'),iris.data, iris.target, cv=5)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [9]:
cross_val_score(svm.SVC(kernel='rbf',C=20,gamma='auto'),iris.data, iris.target, cv=5)

array([0.96666667, 1.        , 0.9       , 0.96666667, 1.        ])

#### Above approach is tiresome and very manual. We can use for loop as an alternative

In [18]:
import numpy as np
kernels = ['rbf', 'linear']
C = [1,10,20]
avg_scores = {}
for kval in kernels:
    for cval in C:
        cv_scores = cross_val_score(svm.SVC(kernel=kval,C=cval,gamma='auto'),iris.data, iris.target)
        avg_scores[kval + '_' + str(cval)] = np.average(cv_scores)
avg_scores

{'rbf_1': 0.9800000000000001,
 'rbf_10': 0.9800000000000001,
 'rbf_20': 0.9666666666666668,
 'linear_1': 0.9800000000000001,
 'linear_10': 0.9733333333333334,
 'linear_20': 0.9666666666666666}

### From above results we can say that rbf with C=1 or 10 or linear with C=1 will give best performance

# Approach 3: Use GridSearchCV

### GridSearchCV does exactly same thing as for loop above but in a single line of code

In [19]:
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(svm.SVC(gamma='auto'), 
                   {
                     'C': [1,10,20], 'kernel': ['rbf','linear']
                    }, cv=5, return_train_score=False)
clf.fit(iris.data, iris.target)
clf.cv_results_

{'mean_fit_time': array([0.00160551, 0.00139856, 0.0016077 , 0.00100036, 0.00100017,
        0.00079918]),
 'std_fit_time': array([4.94098424e-04, 4.89396237e-04, 4.82476867e-04, 6.33466857e-04,
        2.37462990e-06, 3.99591046e-04]),
 'mean_score_time': array([0.00079241, 0.00080028, 0.00099978, 0.00100021, 0.00040202,
        0.00039968]),
 'std_score_time': array([3.96234940e-04, 4.00147203e-04, 1.41934167e-06, 1.09406155e-03,
        4.92447360e-04, 4.89512062e-04]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'ker

In [20]:
df = pd.DataFrame(clf.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001606,0.000494,0.000792,0.000396,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.001399,0.000489,0.000800,0.000400,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.001608,0.000482,0.001000,0.000001,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.001000,0.000633,0.001000,0.001094,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4
4,0.001000,0.000002,0.000402,0.000492,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,5
5,0.000799,0.000400,0.000400,0.000490,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,6


In [21]:
df[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


In [22]:
clf.best_params_

{'C': 1, 'kernel': 'rbf'}

In [23]:
clf.best_score_

0.9800000000000001

### Use RandomizedSearchCV to reduce number of iterations and with random combination of parameters.This is useful when you have too many parameters to try and your training time is longer. It helps reduce the cost of computation

In [24]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(svm.SVC(gamma='auto'), {
     'C': [1,10,20],
     'kernel': ['rbf','linear']
     }, 
 cv=5, 
 return_train_score=False, 
 n_iter=2 )
rs.fit(iris.data, iris.target)
pd.DataFrame(rs.cv_results_)[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,10,linear,0.973333
1,10,rbf,0.980000


## How about different models with different hyperparameters?

In [25]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
model_params = {
 'svm': {
     'model': svm.SVC(gamma='auto'),
     'params' : {
     'C': [1,10,20],
     'kernel': ['rbf','linear']
     } 
 },
 'random_forest': {
     'model': RandomForestClassifier(),
     'params' : {
     'n_estimators': [1,5,10]
     }
 },
 'logistic_regression' : {
     'model': LogisticRegression(solver='liblinear',multi_class='auto'),
     'params': {
     'C': [1,5,10]
    }
 }
}

In [27]:
#use grid search
scores = []
for model_name, mp in model_params.items():
    clf = GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(iris.data, iris.target)
    scores.append({
      'model': model_name,
      'best_score': clf.best_score_,
      'best_params': clf.best_params_
      })
 
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.966667,{'n_estimators': 10}
2,logistic_regression,0.966667,{'C': 5}


In [28]:
#use Randomizesearch
scores = []
for model_name, mp in model_params.items():
     clf = RandomizedSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
     clf.fit(iris.data, iris.target)
     scores.append({
         'model': model_name,
         'best_score': clf.best_score_,
         'best_params': clf.best_params_
 })
 
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

E:\Users\azmina.vanzara\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:278: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
E:\Users\azmina.vanzara\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:278: UserWarning: The total space of parameters 3 is smaller than n_iter=10. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
E:\Users\azmina.vanzara\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:278: UserWarning: The total space of parameters 3 is smaller than n_iter=10. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


,model,best_score,best_params
0,svm,0.980000,"{'kernel': 'rbf', 'C': 1}"
1,random_forest,0.960000,{'n_estimators': 10}
2,logistic_regression,0.966667,{'C': 5}


### Based on above, I can conclude that SVM with C=1 and kernel='rbf' is the best model for solving my problem of iris flower classification

# Example 2

In [29]:
from sklearn import datasets
digits = datasets.load_digits()

In [30]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier

In [31]:
model_params = {
 'svm': {
     'model': svm.SVC(gamma='auto'),
     'params' : {
     'C': [1,10,20],
     'kernel': ['rbf','linear']
     } 
    },
 'random_forest': {
     'model': RandomForestClassifier(),
     'params' : {
     'n_estimators': [1,5,10]
     }
 },
 'logistic_regression' : {
     'model': LogisticRegression(solver='liblinear',multi_class='auto'),
     'params': {
     'C': [1,5,10]
     }
 },
 'naive_bayes_gaussian': {
     'model': GaussianNB(),
     'params': {}
 },
 'naive_bayes_multinomial': {
     'model': MultinomialNB(),
     'params': {}
 },
 'decision_tree': {
     'model': DecisionTreeClassifier(),
     'params': {
     'criterion': ['gini','entropy'],
 
     }
 } 
}

In [32]:
from sklearn.model_selection import GridSearchCV
import pandas as pd
scores = []
for model_name, mp in model_params.items():
     clf = GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
     clf.fit(digits.data, digits.target)
     scores.append({
     'model': model_name,
     'best_score': clf.best_score_,
     'best_params': clf.best_params_
     })
 
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.947697,"{'C': 1, 'kernel': 'linear'}"
1,random_forest,0.899318,{'n_estimators': 10}
2,logistic_regression,0.922114,{'C': 1}
3,naive_bayes_gaussian,0.806928,{}
4,naive_bayes_multinomial,0.870350,{}
5,decision_tree,0.809146,{'criterion': 'entropy'}
